<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

`flaml.autogen` offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framwork allows tool use and human participance through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/FLAML/docs/Use-Cases/Autogen#agents).

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
%%capture --no-stderr
# %pip install flaml[autogen]~=2.0.1

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [4]:
from flaml import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    # filter_dict={
        # "model": ["gpt-4-32k-0314", "gpt-4-32k-v0314"],
        # "model": ["gpt-4", "gpt-4-0314"],
        # "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    # },
)
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
            "gpt",
        },
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [5]:
gpt4_config = {
    "seed": 44,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
}
gpt3_config = {
    "seed": 44,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt35,
}
human = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
#    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=gpt4_config,
)
coder.update_system_message(coder.system_message + '''
Available functions you can import from utils:

def find_papers_pmc(query, size=10):
    """
    Find papers in PubMed Central (PMC) using Europe PMC API.

    Args:
        query (str): The search query to use.
        size (int): The number of results to return (default 10).

    Returns:
        None. Prints the title, url, publication date, and abstract of each paper found.
    """

def search_arxiv(query, max_results=10):
    """
    Searches arXiv for the given query and returns the search results.

    Args:
        query (str): The search query.
        max_results (int, optional): The maximum number of search results to return. Defaults to 10.

    Returns:
        jresults (list): A list of dictionaries. Each dictionary contains fields such as 'title', 'authors', 'summary', and 'pdf_url'

    Example:
        >>> results = search_arxiv("attention is all you need")
        >>> print(results)
    """ 
''')
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You are able to categorize papers after seeing their abstracts."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Interact with the admin to decide the plan and obtain approval.
Explain the plan first. Be clear which step uses a coder, and which step uses a scientist.
Make the result verifiable by including verifiable URL.
Available functions the coder can import from utils:

def find_papers_pmc(query, size=10):
    """
    Find papers in PubMed Central (PMC) using Europe PMC API.

    Args:
        query (str): The search query to use.
        size (int): The number of results to return (default 10).

    Returns:
        None. Prints the title, url, publication date, and abstract of each paper found.
    """

def search_arxiv(query, max_results=10):
    """
    Searches arXiv for the given query and returns the search results.

    Args:
        query (str): The search query.
        max_results (int, optional): The maximum number of search results to return. Defaults to 10.

    Returns:
        jresults (list): A list of dictionaries. Each dictionary contains fields such as 'title', 'authors', 'summary', and 'pdf_url'

    Example:
        >>> results = search_arxiv("attention is all you need")
        >>> print(results)
    """ 
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Execute the code and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "med"},
)
groupchat = autogen.GroupChat(agents=[human, coder, scientist, planner, executor], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Start Chat

In [6]:
human.initiate_chat(
    manager,
    message="""
find 20 latest papers on usage of generative AI in medicine and create a markdown table of different domains where it is being used.
""",
)

Admin (to chat_manager):


find 20 latest papers on usage of generative AI in medicine and create a markdown table of different domains where it is being used.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Sure, here is the plan:

1. **Coder**: Use the `find_papers_pmc` function to find the 20 latest papers on the usage of generative AI in medicine. The query will be "generative AI in medicine". Set the size to 20.

2. **Scientist**: Review the papers found and identify the different domains where generative AI is being used in medicine.

3. **Coder**: Create a markdown table with the following columns: Paper Title, Authors, Publication Date, Domain, and URL. Fill in the Paper Title, Authors, Publication Date, and URL columns with the information from the papers found.

4. **Scientist**: Fill in the Domain column with the information identified in step 2.

5. **Coder**: Provide the markdown table.

The result will be veri